<a href="https://colab.research.google.com/github/trinadh8247/NLP-practice/blob/main/kindle_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
import numpy as np

In [23]:
data=pd.read_csv('/content/all_kindle_review .csv')

In [24]:
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


## best practices
1. prerpocessing and cleaning
2. train test split
3. bow tfidf word2vec
4. train ml algorithms

In [25]:
data=data[['reviewText','rating']]
data.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [26]:
data.isna().sum()

,0
reviewText,0
rating,0


In [27]:
data['rating'].unique()

array([3, 5, 4, 2, 1])

In [28]:
data['rating'].value_counts()

,count
rating,
5,3000
4,3000
3,2000
2,2000
1,2000


In [29]:
## preprocessing and cleanning


In [30]:
data['rating']=data['rating'].apply(lambda x: 0 if x<3 else 1)

In [31]:
data.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [32]:
data['rating'].value_counts()

,count
rating,
1,8000
0,4000


In [33]:
## 1. lower all the cases
data['reviewText']=data['reviewText'].apply(lambda x: x.lower())

In [34]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

## remove all special chars
data['reviewText'] = data['reviewText'].apply(lambda x: re.sub(r'[^a-z A-Z 0-9 ]', '', x))

## remove the stopwords
stop_words = set(stopwords.words('english'))
data['reviewText'] = data['reviewText'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

## remove url
data['reviewText'] = data['reviewText'].apply(lambda x: re.sub(r'http\S+|www.\S+', '', x))

## remove html tags
data['reviewText'] = data['reviewText'].apply(lambda x: re.sub(r'<.*?>', '', x))

## remove any additional spaces
data['reviewText'] = data['reviewText'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
data.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [36]:
## applying lematization
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [37]:
def lemmatize_words(text):
  return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [38]:
data['reviewText'] = data['reviewText'].apply(lambda x:lemmatize_words(x))

In [40]:
data.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [41]:
## train test spslit
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data['reviewText'],data['rating'],test_size=0.2,random_state=42)

In [42]:
##BOW impementation

In [46]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
X_train_bow=cv.fit_transform(X_train).toarray()
X_test_bow=cv.transform(X_test).toarray()

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X_train_tfidf=tfidf.fit_transform(X_train).toarray()
X_test_tfidf=tfidf.transform(X_test).toarray()

In [47]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,classification_report

In [48]:
mnomial_bow=MultinomialNB().fit(X_train_bow,y_train)
mnomial_tfidf=MultinomialNB().fit(X_train_tfidf,y_train)

In [49]:
y_pred_bow=mnomial_bow.predict(X_test_bow)
y_pred_tfidf=mnomial_tfidf.predict(X_test_tfidf)

In [50]:
print('bow :',accuracy_score(y_test,y_pred_bow))
print('tfidf :',accuracy_score(y_test,y_pred_tfidf))

bow : 0.8341666666666666
tfidf : 0.6941666666666667


In [52]:
print(classification_report(y_test,y_pred_bow))
print(classification_report(y_test,y_pred_tfidf))

              precision    recall  f1-score   support

           0       0.81      0.66      0.73       803
           1       0.84      0.92      0.88      1597

    accuracy                           0.83      2400
   macro avg       0.83      0.79      0.80      2400
weighted avg       0.83      0.83      0.83      2400

              precision    recall  f1-score   support

           0       0.99      0.09      0.16       803
           1       0.69      1.00      0.81      1597

    accuracy                           0.69      2400
   macro avg       0.84      0.54      0.49      2400
weighted avg       0.79      0.69      0.59      2400

